# 합성곱 신경망(CNN)을 이용한 이미지 분류

**학습 목표:**
- 이미지 데이터의 공간적 특징(spatial feature)을 효과적으로 학습하는 **합성곱 신경망(CNN)**의 구조를 이해합니다.
- CNN의 핵심 구성 요소인 **합성곱(Convolution)**, **활성화 함수(ReLU)**, **풀링(Pooling)** 계층의 역할과 동작 방식을 학습합니다.
- **PyTorch**를 사용하여 직접 CNN 모델을 구축하고, 10개의 클래스로 구성된 **CIFAR-10** 이미지 데이터셋을 분류하는 실습을 진행합니다.
- **데이터 증강(Data Augmentation)**을 통해 모델의 일반화 성능을 높이는 기법을 적용합니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

### (1) 하이퍼파라미터 및 장치 설정

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

batch_size = 128
learning_rate = 0.001
epochs = 20

### (2) 데이터 준비: CIFAR-10
CIFAR-10 데이터셋은 `비행기, 자동차, 새, 고양이, 사슴, 개, 개구리, 말, 배, 트럭` 10개 클래스로 구성된 3x32x32 크기의 컬러 이미지입니다. 모델이 다양한 이미지에 강건해지도록, 훈련 데이터에만 **데이터 증강(Data Augmentation)** 기법(이미지를 무작위로 자르거나 뒤집기)을 적용합니다.

In [ ]:
# 훈련 데이터용 변환: 데이터 증강 포함
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # CIFAR-10 데이터의 평균과 표준편차
])

# 테스트 데이터용 변환: 데이터 증강 없음
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 데이터 샘플 시각화
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images[:8]))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(8)))

### (3) 모델 정의: BasicCNN
두 개의 합성곱 블록(Conv-ReLU-Pool)과 세 개의 완전 연결 계층(MLP)으로 구성된 간단한 CNN 모델을 정의합니다. 합성곱 층은 이미지의 특징을 추출하고, 풀링 층은 특징 맵의 크기를 줄여 계산 효율성을 높이고 주요 특징을 강조하는 역할을 합니다.

In [ ]:
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.classifier(x)
        return x

model = BasicCNN().to(device)

### (4) 손실 함수, 옵티마이저, 학습률 스케줄러 설정
- 옵티마이저는 `Adam`을 사용합니다.
- `StepLR` 스케줄러는 특정 단계마다 학습률을 감소시켜, 훈련 후반부에 모델이 최적점에 더 안정적으로 수렴하도록 돕습니다.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # 7 에포크마다 학습률 0.1배 감소

### (5) 모델 훈련 및 평가

In [ ]:
train_losses = []
test_accuracies = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_losses.append(running_loss / len(train_loader))
    scheduler.step() # 스케줄러 업데이트

    # 에포크마다 테스트 정확도 확인
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    test_accuracies.append(accuracy)
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_losses[-1]:.4f}, Test Accuracy: {accuracy:.2f}%')

### (6) 학습 과정 시각화

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(train_losses)
ax1.set_title("Training Loss")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")

ax2.plot(test_accuracies)
ax2.set_title("Test Accuracy")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy (%)")

plt.show()